In [1]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split

# GPU support
DEVICE = torch.device('cuda') # cuda
shuffle=True
batch_size = 32

In [2]:
path_sys = '/media/sider/data' if os.path.exists('/media/sider/data') else '/home/smartrue'
path_data = os.path.join(path_sys, 'Dropbox/current_codes/PycharmProjects/AdmieRP_train/DATA')

parks = joblib.load(os.path.join(path_data, 'parks_static_info.pickle'))

In [52]:
rated = np.array([p['rated'] for p in parks])
rated

array([38.      ,  9.35    , 18.7     , 16.4724  , 28.3521  , 14.5383  ,
       16.1     , 41.2986  , 28.8     , 14.4     ,  4.8846  , 16.      ,
       20.24505 , 10.2648  ,  2.2848  , 22.64325 , 10.19235 , 10.899   ,
        8.8809  , 16.9785  , 19.3809  , 15.      , 21.00105 , 14.55    ,
       18.      ,  3.2361  , 24.3     , 14.4     , 14.4     , 58.      ,
       18.      ,  3.57    , 42.      , 17.50455 ,  3.44505 , 43.26735 ,
        6.5247  ,  3.22455 , 10.82025 , 24.51015 , 18.91155 , 24.4335  ,
        8.4     , 38.183775, 70.98315 , 68.7015  , 42.      , 42.      ,
       27.      , 21.15    , 18.8     , 42.3     , 42.903   ,  8.4     ,
       18.      , 24.      , 12.0771  , 65.4     , 54.      , 37.35375 ,
       17.      , 17.1738  , 28.8     ,  5.25    , 28.8     , 34.5807  ,
       27.6     , 52.23855 , 50.      , 41.8     ,  4.3554  , 10.8     ,
       26.9031  , 21.6     , 10.8     , 35.007   , 12.      ,  3.60465 ,
       40.      ,  7.2     ,  5.4     , 89.43795 , 

<h1>
<center>A3T-GCN: Attention Temporal Graph
Convolutional Network for Traffic Forecasting</center>
</h1>


## Dataset
- Traffic forecasting dataset based on Los Angeles Metropolitan traffic 
- 207 loop detectors on highways
- March 2012 - June 2012
- From the paper: Diffusion Convolutional Recurrent Neural Network


In [37]:
spatial = []
for i, park in enumerate(parks):
    df = pd.DataFrame([[park['name'], i, park['lat'], park['long']]],
                      columns=['name', 'id', 'latitude', 'longitude'])
    spatial.append(df)
spatial = pd.concat(spatial, ignore_index=True)
spatial.set_index('name', inplace=True)
edges = []
edges_weights = []
for i, park in enumerate(parks):
    coordinates = spatial.loc[park['name']]
    variables = park['variables']
    variables = pd.DataFrame(variables, columns=['id', 'name']).sort_values(by='id')['name'].values
    for feature in variables:
        tag = feature.split('_')[-1]
        name = '_'.join(feature.split('_')[:-1])
        coord_temp = spatial.loc[name]
        edges.append([i, coord_temp['id']])
        edges_weights.append(np.sqrt((coordinates['latitude'] - coord_temp['latitude'])**2 + (coordinates['longitude'] - coord_temp['longitude'])**2))

In [38]:
edges = np.array(edges).T
edges_weights = np.array(edges_weights).T

In [5]:
from torch_geometric_temporal.dataset.admie_ds import ADMIEDataset


from ADMIE.configuration.config_input_data import (variables, TARGET_VARIABLE)
variables_template = variables()
dataset_train = ADMIEDataset(
            path_data,
            variables_template,
            TARGET_VARIABLE,
            rated,
            task='train')
dataset_val = ADMIEDataset(
            path_data,
            variables_template,
            TARGET_VARIABLE,
            rated,
            task='val')
dataset_test = ADMIEDataset(
            path_data,
            variables_template,
            TARGET_VARIABLE,
            rated,
            task='test')
print("Dataset type:  ", dataset_train)
print(next(iter(dataset_train))) # Show first sample

100%|██████████| 280447/280447 [00:06<00:00, 40321.20it/s]

Dataset type:   <torch_geometric_temporal.dataset.admie_ds.ADMIEDataset object at 0x718077291f10>
(tensor([[[0.0000e+00, 5.6944e+00, 5.6944e+00,  ..., 6.3953e+00,
          6.4637e+00, 6.6574e+00],
         [0.0000e+00, 1.6319e+02, 1.6319e+02,  ..., 1.6723e+02,
          1.6846e+02, 1.6846e+02]],

        [[0.0000e+00, 5.9897e+00, 5.9897e+00,  ..., 3.0085e+00,
          2.3932e+00, 2.1197e+00],
         [0.0000e+00, 3.3618e+01, 3.0382e+01,  ..., 2.2576e+01,
          1.0878e+01, 5.8081e+01]],

        [[0.0000e+00, 8.0000e+00, 8.0000e+00,  ..., 4.0342e+00,
          4.1618e+00, 4.0205e+00],
         [0.0000e+00, 3.3196e+01, 3.5587e+01,  ..., 1.6387e+01,
          1.2284e+01, 5.4048e+01]],

        ...,

        [[0.0000e+00, 8.9829e+00, 8.9829e+00,  ..., 6.9829e+00,
          6.9829e+00, 6.4496e+00],
         [0.0000e+00, 8.0917e+01, 7.0555e+01,  ..., 7.9874e+01,
          8.3542e+01, 8.4687e+01]],

        [[0.0000e+00, 1.2308e+00, 0.0000e+00,  ..., 1.2308e-01,
          0.0000e+00, 8

# Creating DataLoaders


In [6]:
from torch.utils.data import DataLoader
dataloader_train = DataLoader(dataset_train, shuffle=shuffle, batch_size=batch_size)
dataloader_val = DataLoader(dataset_val, shuffle=shuffle, batch_size=batch_size)
dataloader_test = DataLoader(dataset_test, shuffle=shuffle, batch_size=batch_size)

In [7]:
past, future, target = next(iter(dataloader_train))

In [8]:
past.shape


torch.Size([32, 179, 2, 21])

## Model

Which model to choose depends on which time-series task you work on. 

- A3TGCN is an extension of TGCN that uses attention 
- The spatial aggregation uses GCN, the temporal aggregation a GRU
- We can pass in periods to get an embedding for several timesteps
- This embedding can be used to predict several steps into the future = output dimension
- We could also do this in a loop and feed it again into the model (would be autoregressive)
- There is only one block here. Other layers also allow stacking???

<html>
<img src="https://i.ibb.co/WxrJQbc/a3tgcn.png", height="300"></img>

# TGCN model
A temporal GCN (T-GCN) model was constructed by combining GCN and GRU. 

n historical time series traffic data were inputted into the T-GCN model to obtain n hidden states (h) that covered spatiotemporal characteristics:{h(t−n), · · · , h(t−1), h(t)}


ut = σ(Wu ∗ (GC(A, Xt), ht−1)) 

rt = σ(Wr ∗ (GC(A, Xt), ht−1)) 

ct = tanh(Wc ∗ (GC(A, Xt), (rt ∗ ht−1)))

ht = ut ∗ ht−1 + (1 − ut) ∗ ct) 


Then, the hidden states were inputted into the attention model to determine the context vector that covers the global traffic variation information. Particularly, the weight of each h was calculated by Softmax using a multilayer perception:{at−n, · · · , at−1, at}.The context vector that covers global traffic variation information is calculated by the weighted sum. 


# A3TGCN Model
The A3TGCN is an extention of the TGCN model by adding an attention mechanism.

The attention mechanism was introduced to re-weight the influence of historical traffic states and thus to capture the global variation trends of traffic state

In [46]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.tgnn = A3TGCN2(in_channels=node_features,  out_channels=32, periods=periods,batch_size=batch_size) # node_features=2, periods=12
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(32, periods)

    def forward(self, x, edge_index, edge_weights):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index, edge_weights) # x [b, 207, 2, 12]  returns h [b, 207, 12]
        h = F.relu(h) 
        h = self.linear(h)
        return h


In [47]:
class BiTemporalGNN(torch.nn.Module):
    def __init__(self,):
        super(BiTemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.bitgnn1 = TemporalGNN(node_features=2, periods=21, batch_size=128) # node_features=2, periods=12
        self.bitgnn2 = TemporalGNN(node_features=2, periods=20, batch_size=128) # node_features=2, periods=12
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(64, 1)

    def forward(self, past, future, edge_index, edges_weights):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h1 = self.bitgnn1(past, edge_index, edges_weights) # x [b, 207, 2, 12]  returns h [b, 207, 12]
        h2 = self.bitgnn2(future, edge_index, edges_weights) # x [b, 207, 2, 12]  returns h [b, 207, 12]
        h = F.relu(torch.cat([h1, h2], dim=-1))
        h = self.linear(h)
        return h

## Training

In [48]:

# Create model and optimizers
model = BiTemporalGNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()
# edges = torch.from_numpy(edges).type(torch.LongTensor).to(DEVICE)
# edges_weights = torch.from_numpy(edges_weights).type(torch.FloatTensor).to(torch.float32).to(DEVICE)

print('Net\'s state_dict:')
total_param = 0
for param_tensor in model.state_dict():
    print(param_tensor, '\t', model.state_dict()[param_tensor].size())
    total_param += np.prod(model.state_dict()[param_tensor].size())
print('Net\'s total params:', total_param)
#--------------------------------------------------
print('Optimizer\'s state_dict:')
for var_name in optimizer.state_dict():
    print(var_name, '\t', optimizer.state_dict()[var_name])

Net's state_dict:
bitgnn1.tgnn._attention 	 torch.Size([21])
bitgnn1.tgnn._base_tgcn.conv_z.bias 	 torch.Size([32])
bitgnn1.tgnn._base_tgcn.conv_z.lin.weight 	 torch.Size([32, 2])
bitgnn1.tgnn._base_tgcn.linear_z.weight 	 torch.Size([32, 64])
bitgnn1.tgnn._base_tgcn.linear_z.bias 	 torch.Size([32])
bitgnn1.tgnn._base_tgcn.conv_r.bias 	 torch.Size([32])
bitgnn1.tgnn._base_tgcn.conv_r.lin.weight 	 torch.Size([32, 2])
bitgnn1.tgnn._base_tgcn.linear_r.weight 	 torch.Size([32, 64])
bitgnn1.tgnn._base_tgcn.linear_r.bias 	 torch.Size([32])
bitgnn1.tgnn._base_tgcn.conv_h.bias 	 torch.Size([32])
bitgnn1.tgnn._base_tgcn.conv_h.lin.weight 	 torch.Size([32, 2])
bitgnn1.tgnn._base_tgcn.linear_h.weight 	 torch.Size([32, 64])
bitgnn1.tgnn._base_tgcn.linear_h.bias 	 torch.Size([32])
bitgnn1.linear.weight 	 torch.Size([21, 32])
bitgnn1.linear.bias 	 torch.Size([21])
bitgnn2.tgnn._attention 	 torch.Size([20])
bitgnn2.tgnn._base_tgcn.conv_z.bias 	 torch.Size([32])
bitgnn2.tgnn._base_tgcn.conv_z.lin.weigh

# Loading the graph once 
because it's a static graph

In [49]:
model.train()

for epoch in range(1):
    step = 0
    loss_list = []
    for past, future, labels in dataloader_train:
        past = past.to(torch.float32).to(DEVICE)
        future = future.to(torch.float32).to(DEVICE)
        labels = labels.to(torch.float32).to(DEVICE)
        y_hat = model(past, future, edges, edges_weights)         # Get model predictions
        loss = loss_fn(y_hat, labels) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step= step + 1
        loss_list.append(loss.item())
        if step % 100 == 0 :
            print(sum(loss_list)/len(loss_list))
    print("Epoch {} train RMSE: {:.4f}".format(epoch, sum(loss_list)/len(loss_list)))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5728x41 and 64x1)

## Evaluation

- Lets get some sample predictions for a specific horizon (e.g. 288/12 = 24 hours)
- The model always gets one hour and needs to predict the next hour

In [ ]:
model.eval()
step = 0
# Store for analysis
total_loss = []
for past, future, labels in dataloader_train:
    past = past.to(torch.float32).to(DEVICE)
    future = future.to(torch.float32).to(DEVICE)
    labels = labels.to(torch.float32).to(DEVICE)
    y_hat = model(past, future, edges, edges_weights)
    # Mean squared error
    loss = loss_fn(y_hat, labels)
    total_loss.append(loss.item())
    # Store for analysis below
    #test_labels.append(labels)
    #predictions.append(y_hat)
    

print("Test MSE: {:.4f}".format(sum(total_loss)/len(total_loss)))

### Visualization

- The further away the point in time is, the worse the predictions get
- Predictions shape: [num_data_points, num_sensors, num_timesteps]

In [ ]:
sensor = 123
timestep = 11 
preds = np.asarray([pred[sensor][timestep].detach().cpu().numpy() for pred in y_hat])
labs  = np.asarray([label[sensor][timestep].cpu().numpy() for label in labels])
print("Data points:,", preds.shape)

In [ ]:
plt.figure(figsize=(20,5))
sns.lineplot(data=preds, label="pred")
sns.lineplot(data=labs, label="true")